# LangChain 1.0 Dynamic Model Selection

Ref:
https://docs.langchain.com/oss/python/langchain/agents#dynamic-model

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 mlflow

%restart_python

## モデルとの接続

In [0]:
from langchain.chat_models import init_chat_model
import mlflow


mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
llm_config = {"api_key": creds.token, "base_url": creds.host + "/serving-endpoints"}

# 2種のモデルを準備
basic_model = init_chat_model("openai:databricks-gpt-oss-20b", **llm_config)
advanced_model = init_chat_model("openai:databricks-gpt-oss-120b", **llm_config)

## エージェントの作成と実行

In [0]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """会話の複雑さに基づいてモデルを選択します。"""
    message_count = len(request.state["messages"])

    if message_count > 3:
        # 処理履歴が長い場合は高度なモデルを使用する
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

In [0]:
from langchain.agents import create_agent
from pprint import pprint

def get_weather(city: str) -> str:
    """指定した都市の天気を取得します。"""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=basic_model,
    tools=[get_weather],
    middleware=[dynamic_model_selection],
)

print("---- simple ---")
input = {"messages": [{"role": "user", "content": "SFの天気は？"}]}
result = agent.invoke(input)
print("answer:", result["messages"][-1].content)
print("model:", result["messages"][-1].response_metadata["model_name"])

print()

print("---- complex ---")
input = {"messages": [{"role": "user", "content": "東京・大阪・福岡の天気を教えてください。"}]}
result = agent.invoke(input)
print("answer:", result["messages"][-1].content)
print("model:", result["messages"][-1].response_metadata["model_name"])
